<a href="https://colab.research.google.com/github/Master-Chief131/ShopSmart/blob/main/shopsmart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Realizar Scraping**
## Ahora vamos a realizar un script para extraer todos los productos de la pagina de super 99, y lo haremos navegando por las categorias

Como primer paso vamos a ver como clasifica sus productos super 99, para eso vamos a extrar el menu de su pagina principal

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import csv

Primero definimos la url base y parseamos el archivo html para poder navegar por su contenido

In [ ]:
# Pagina de super 99
URL_BASE = "https://www.super99.com"
respuesta_obtenida = requests.get(URL_BASE)
html_home = respuesta_obtenida.text

# Parsear el html
super99_home = BeautifulSoup(html_home, "html.parser")

Ahora debemos buscar el contenido del menu y buscar su enlace para realizar una busqueda de todos los productos por pagina

In [ ]:
categorias = super99_home.find_all("a", class_="level-top")

categoria = []

for enlace in categorias:
    categoria.append(enlace["href"])

Una vez tenemos los enlaces a las paginas debemos ir a cada categoria y obtener sus productos

In [ ]:
# usar el primer enlace
Url_productos = categoria[0]
respuesta_productos = requests.get(Url_productos)
html_productos = respuesta_productos.text

# Parsear el html
super99_productos = BeautifulSoup(html_productos, "html.parser")

Revisamos cuantas paginas tiene esa categoria

In [ ]:
# El enlace a la ultima pagina de esa categoria
paginas = super99_productos.find("a", class_="page last")

# devuelve dos span
spans = paginas.find_all("span")

span_sin_clase = [span for span in spans if not span.get("class")]

# utilizamos el bucle for para extraer el span con el numero y lo convertimos a entero
if span_sin_clase:
    numero_pagina = int(span_sin_clase[0].text.strip())
    print(f"Número de la última página: {numero_pagina}") # Lo usaremos para iterar las paginas
else:
    print("No se encontró un <span> sin clase.")

Primero descarguemos las imagenes

In [ ]:
# Buscar las imagenes
src_productos = super99_productos.find_all("img", class_="product-image-photo")

nombre_productos = []
url_imagenes = []

for i, imagen in enumerate(src_productos):
    url_imagenes.append(imagen["src"])
    nombre_productos.append(imagen["alt"].strip())

In [ ]:
 Las url tienen atributos incrustados
# Limpiar las url
cleaned_urls = [re.sub(r'\?.*', '', url) for url in url_imagenes]
cleaned_names = [re.sub(r"/", "-", re.sub(r" ", "_", name)) for name in nombre_productos]

Para poder distinguir las imagenes las guardaremos con el nombre del producto

In [ ]:
# Descargamos las imagenes y las enumeramos
for i, imagen in enumerate(cleaned_urls):
  nombre_p = cleaned_names[i]
  r = requests.get(imagen)
  with open(f"{nombre_p}.png", "wb") as f:
    f.write(r.content)

Ahora buscamos el nombre y el precio de los productos

In [ ]:
# extraer el nombre y el precio de los productos
producto_info = super99_productos.find_all("div", class_="details-info-product")

producto_nombre = []
producto_precio = []

for info in producto_info:
    nombre = info.find("a", class_="product-item-link").text.strip().replace("/", "-").replace(" ", "_")
    precio = info.find("span", class_="price-wrapper").text.strip().replace("$", "")
    producto_nombre.append(nombre)
    producto_precio.append(precio)

In [ ]:
producto_nombre.insert(0, "Producto")
producto_precio.insert(0, "Precio")

In [ ]:
datos = dict(zip(producto_nombre, producto_precio))

In [ ]:
with open('datos.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    for key, value in datos.items():
        writer.writerow([key, value])